In [1]:
import laserhockey.hockey_env as h_env
import numpy as np
import torch
import time
from sac import SAC_Agent
from dsac import DSAC_Agent

Using device: cpu
Using device: cpu


In [3]:
print(h_env.HockeyEnv().observation_space)

Box(-inf, inf, (18,), float32)


In [2]:
torch.manual_seed(42)
np.random.seed(42)

env = h_env.HockeyEnv()
agent = DSAC_Agent(env.observation_space,env.action_space)
agent.load_network_states(torch.load("./results/DSAC_after_episode10000_easy-e10500-t32-s42-player.pth", map_location=torch.device('cpu')))
agent.eval()
#agent.load_network_states(torch.load("SAC_run_easy-e1500-t32-s42-player.pth"))
opponent = h_env.BasicOpponent(weak=True)

loss = 0
win = 0
tie = 0

for run in range(0,1000):
    ob, info = env.reset()
    ob_opponent = env.obs_agent_two()
    for t in range(1,1000):
        action = agent.act(ob)
        opponent_action = opponent.act(ob_opponent)
        #opponent_action = np.zeros_like(opponent_action)
        ob_new,reward,done,trunc,info = env.step(np.hstack([action,opponent_action]))
        info_opponent = env.get_info_agent_two()
        #if info["reward_touch_puck"]!=0:
        #    print(info)
        #    print("touched")
        #time.sleep(0.01)
        #env.render()
        #print(info["reward_closeness_to_puck"])
        if done:
            if env.winner ==1:
                win +=1
                #if reward > 0:
                #    print(reward)
            elif env.winner == -1: 
                loss +=1
            else:
                tie +=1
            break
        #if reward > 0:
        #    print(reward)

        ob = ob_new
        ob_opponent = env.obs_agent_two()
print(info)
env.close()

print(f"win(left_side): {win},loss: {loss}, tie: {tie}")

Agent now in evaluation Mode
{'winner': 0, 'reward_closeness_to_puck': -0.10243002142950554, 'reward_touch_puck': 0.0, 'reward_puck_direction': 0.0}
win(left_side): 51,loss: 333, tie: 616


In [2]:
import matplotlib.pyplot as plt
import pickle

with open(f"SAC_Easy_easy-s42-e4500-stat.pkl", 'rb') as f:
    data = pickle.load(f)
    rewards = np.asarray(data["rewards"])
    q_losses =  np.asarray(data["q_losses"])
    pi_losses  = np.asarray(data["pi_losses"])
    temperature_losses = np.asarray(data["temperature_loss"])

In [2]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'